This is vertion 2 of our work, changes we made:

1)we created index on 1 mil pages, and the all the terms from the jason file

2)we fixed many bugs and made the search work proprely

3)added another serach function using MB25 didnt test it yet.

4)learned how to read the and write to/from the bucket

5)did not run colab fronted yet, but results from 1 mil seems to not be in the final answer.

In [ ]:
# if the following command generates an error, you probably didn't enable
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

NAME          PLATFORM  PRIMARY_WORKER_COUNT  SECONDARY_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
cluster-9ce1  GCE       4                                             RUNNING  us-central1-a


# Imports & Setup

In [ ]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.6/106.6 kB 3.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 0.21.0 requires google-cloud-storage>=2.0.0, but you have google-cloud-storage 1.43.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 5.3 MB/s eta 0:00:00


In [ ]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

ModuleNotFoundError: No module named 'pyspark'

In [ ]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Mar  2 20:28 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [ ]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [ ]:
spark

In [ ]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = 'bucket-mr-project-ir-david'
#full_path = f"gs://{bucket_name}/wiki_dump/"
full_path = "gs://bucket-mr-project-ir-david/"
paths=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    if b.name.startswith('wiki_dump/') and b.name != 'wiki_dump/graphframes.sh' and b.name != 'wiki_dump/':
        paths.append(full_path + b.name)


base_dir/0_000.bin
base_dir/0_posting_locs.pickle
base_dir/100_000.bin
base_dir/100_posting_locs.pickle
base_dir/101_000.bin
base_dir/101_posting_locs.pickle
base_dir/102_000.bin
base_dir/102_posting_locs.pickle
base_dir/103_000.bin
base_dir/103_posting_locs.pickle
base_dir/104_000.bin
base_dir/104_posting_locs.pickle
base_dir/105_000.bin
base_dir/105_posting_locs.pickle
base_dir/106_000.bin
base_dir/106_posting_locs.pickle
base_dir/107_000.bin
base_dir/107_posting_locs.pickle
base_dir/108_000.bin
base_dir/108_posting_locs.pickle
base_dir/109_000.bin
base_dir/109_posting_locs.pickle
base_dir/10_000.bin
base_dir/10_posting_locs.pickle
base_dir/110_000.bin
base_dir/110_posting_locs.pickle
base_dir/111_000.bin
base_dir/111_posting_locs.pickle
base_dir/112_000.bin
base_dir/112_posting_locs.pickle
base_dir/113_000.bin
base_dir/113_posting_locs.pickle
base_dir/114_000.bin
base_dir/114_posting_locs.pickle
base_dir/115_000.bin
base_dir/115_posting_locs.pickle
base_dir/116_000.bin
base_dir/116_

***GCP setup is complete!*** If you got here without any errors you've earned 10 out of the 35 points of this part.

# Building an inverted index

buliding inverted on 1000000 docs and the terms only from the jason file


In [ ]:
# take the 'text' and 'id' or the first 1000 rows and create an RDD from it
#remove this later
query_tokens = ["genetics", "Who", "is", "considered", "the", "Father", "of", "United", "States", "economic", "When", "was", "Nations", "founded", "video", "gaming", "D", "printing", "technology", "author", "bioinformatics", "known", "for", "proposing", "heliocentric", "model", "solar", "system", "Describe", "process", "water", "erosion", "Berlin", "Wall", "constructed", "What", "meaning", "term", "Habeas", "Corpus", "telecommunications", "internet", "characteristics", "a", "chemical", "element", "structure", "plant", "cell", "painted", "Starry", "Night", "computer", "Earth", "layers", "did", "World", "War", "II", "end", "Gutenberg", "press", "invented", "medicine", "cycle", "artificial", "intelligence", "physics", "nanotechnology", "Black", "Death", "pandemic", "occur", "neuroscience", "snowboard", "founder", "modern", "psychology"]
parquetFile = spark.read.parquet(*paths)
doc_text_pairs = parquetFile.limit(1000000).select("text", "id").rdd
doc_title_pairs = parquetFile.limit(1000000).select("id", "title").rdd
print(doc_title_pairs.take(1))

[Row(id=9172811, title='Brian Osborne')]


We will count the number of pages to make sure we are looking at the entire corpus. The number of pages should be more than 6M

In [ ]:
# Count number of wiki pages
parquetFile.count()

6348910

Let's import the inverted index module. Note that you need to use the staff-provided version called `inverted_index_gcp.py`, which contains helper functions to writing and reading the posting files similar to the Colab version, but with writing done to a Google Cloud Storage bucket.

In [ ]:
# if nothing prints here you forgot to upload the file inverted_index_gcp.py to the home dir
%cd -q /home/dataproc
!ls inverted_index_gcp.py

inverted_index_gcp.py


In [ ]:
# adding our python module to the cluster
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

In [ ]:
from inverted_index_gcp import InvertedIndex

all the logic

In [ ]:
NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS
bucket_name = "bucket-mr-project-ir-david"
base_dir = "index_text_on_jason"
sum_doc_len = 0

english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links",
                    "may", "first", "see", "history", "people", "one", "two",
                    "part", "thumb", "including", "second", "following",
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

# PLACE YOUR CODE HERE

def word_count(text, id):
  ''' Count the frequency of each word in `text` (tf) that is not included in
  `all_stopwords` and return entries that will go into our posting lists.
  Parameters:
  -----------
    text: str
      Text of one document
    id: int
      Document id
  Returns:
  --------
    List of tuples
      A list of (token, (doc_id, tf)) pairs
      for example: [("Anarchism", (12, 5)), ...]
  '''
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  len_of_doc = len(tokens)
  # YOUR CODE HERE
  word_counts = Counter(tokens)
    # Filter out stopwords and create tuples of (token, (doc_id, tf))
  result = [(token, (id, tf)) for token, tf in word_counts.items() if token not in all_stopwords and token in query_tokens]#remove and token in query_tokens
  return result

def doc_len(text, id):
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  tokens_without_stop_w = [token for token in tokens if token not in all_stop_words)]
  sum_doc_len += len(tokens)
  return id,len(tokens)

def reduce_word_counts(unsorted_pl):
  ''' Returns a sorted posting list by wiki_id.
  Parameters:
  -----------
    unsorted_pl: list of tuples
      A list of (wiki_id, tf) tuples
  Returns:
  --------
    list of tuples
      A sorted posting list.
  '''
  # YOUR CODE HERE
  # Sort the posting list by wiki_id
  sorted_pl = sorted(unsorted_pl, key=lambda x: x[0])
  return sorted_pl

def calculate_df(postings):
  ''' Takes a posting list RDD and calculate the df for each token.
  Parameters:
  -----------
    postings: RDD
      An RDD where each element is a (token, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each element is a (token, df) pair.
  '''
  # YOUR CODE HERE
  # Calculate the length of each posting list for each token
  token_document_counts = postings.map(lambda x: (x[0], len(x[1])))

  return token_document_counts

def partition_postings_and_write(postings):
  ''' A function that partitions the posting lists into buckets, writes out
  all posting lists in a bucket to disk, and returns the posting locations for
  each bucket. Partitioning should be done through the use of `token2bucket`
  above. Writing to disk should use the function  `write_a_posting_list`, a
  static method implemented in inverted_index_colab.py under the InvertedIndex
  class.
  Parameters:
  -----------
    postings: RDD
      An RDD where each item is a (w, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each item is a posting locations dictionary for a bucket. The
      posting locations maintain a list for each word of file locations and
      offsets its posting list was written to. See `write_a_posting_list` for
      more details.
  '''
  # YOUR CODE HERE
  def write_partition_to_disk(partition):
        ''' Writes a partition of posting lists to disk and returns the posting
        locations for each word in the partition.
        Parameters:
        -----------
            partition: iterator
                An iterator of (bucket_id, posting_lists) tuples.
        Returns:
        --------
            tuple
                A tuple containing the bucket ID and the posting locations dictionary
                for the partition.
        '''
        bucket_id, posting_lists = partition
        bucket_id , posting_locs = InvertedIndex.write_a_posting_list((bucket_id, posting_lists),base_dir,bucket_name)
        return posting_locs
  # Partition the posting lists into buckets
  bucketed_postings = postings.map(lambda x: (token2bucket_id(x[0]), [x])) \
                               .reduceByKey(lambda a, b: a + b)

    # Write out each bucket to disk and collect the posting locations
  posting_locations = bucketed_postings.map(write_partition_to_disk)

  return posting_locations



############################################# page rank
def generate_graph(pages):
  ''' Compute the directed graph generated by wiki links.
  Parameters:
  -----------
    pages: RDD
      An RDD where each row consists of one wikipedia articles with 'id' and
      'anchor_text'.
  Returns:
  --------
    edges: RDD
      An RDD where each row represents an edge in the directed graph created by
      the wikipedia links. The first entry should the source page id and the
      second entry is the destination page id. No duplicates should be present.
    vertices: RDD
      An RDD where each row represents a vetrix (node) in the directed graph
      created by the wikipedia links. No duplicates should be present.
  '''
  # YOUR CODE HERE

  # Extract edges from the 'id' and 'anchor_text' fields of the RDD
  edges = pages.flatMap(lambda row: [(row.id, anchor.id) for anchor in row.anchor_text])

  # Remove duplicates from edges
  edges = edges.distinct()

  vertices = edges.flatMap(lambda edge: ([(edge[0],edge[0]),(edge[1],edge[1])])).distinct()


  return edges, vertices


In [ ]:
# time the index creation time
t_start = time()

#create dict for len of docs
dict_len_docs_rdd = doc_text_pairs.map(lambda x: doc_len(x[0], x[1]))

# Collect the RDD as a dictionary
dict_len_docs = dict_len_docs_rdd.collectAsMap()
# word counts map
word_counts = doc_text_pairs.flatMap(lambda x: word_count(x[0], x[1]))
postings = word_counts.groupByKey().mapValues(reduce_word_counts)
# filtering postings and calculate df
postings_filtered = postings.filter(lambda x: len(x[1])>15)
w2df = calculate_df(postings_filtered)

w2df_dict = w2df.collectAsMap()

dict_doc_title = doc_title_pairs.collectAsMap()

# partition posting lists and write out
posting_locs_list = partition_postings_and_write(postings_filtered).collect()


# collect all posting lists locations into one super-set
super_posting_locs = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='index_text_on_jason'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs[k].extend(v)


# pages_links = spark.read.parquet(path).limit(1000).select("id", "anchor_text").rdd
# edges, vertices = generate_graph(pages_links)
# edgesDF = edges.toDF(['src', 'dst']).repartition(4, 'src')
# verticesDF = vertices.toDF(['id']).repartition(4, 'id')
# g = GraphFrame(verticesDF, edgesDF)
# pr_results = g.pageRank(resetProbability=0.15, maxIter=10)
# pr = pr_results.vertices.select("id", "pagerank")
# pr = pr.sort(col('pagerank').desc())
# pr.repartition(1).write.csv('pr', compression="gzip")
# pr.show()

# Create inverted index instance
inverted = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted.posting_locs = super_posting_locs
# Add the token - df dictionary to the inverted index
inverted.df = w2df_dict
inverted.document_len = dict_len_docs
inverted.doc_title_dict = dict_doc_title
inverted.avg_doc_len = sum_doc_len/len(inverted.document_len)
# write the global stats out
inverted.write_index(base_dir, 'inverted_index_small',bucket_name)
# # upload to gs
# index_src = "index.pkl"
# index_dst = f'gs://{bucket_name}/postings_gcp/{index_src}'
# !gsutil cp $index_src $index_dst

index_const_time = time() - t_start

In [ ]:
# test index construction time
print(index_const_time)
assert index_const_time < 60*120

959.547914981842


upload the index from the bucket


In [ ]:
# load_from_bucket
# get textIndex.pkl from bucket
from google.cloud import storage
import pickle
bucket_name = 'bucket-mr-project-ir-david'
file_path = "index_text_on_jason/inverted_index_small.pkl"
storage_client = storage.Client()
bucket = storage_client.bucket(bucket_name)
blob = bucket.blob(file_path)
contents = blob.download_as_bytes()
inverted = pickle.loads(contents)

search and vectors

In [ ]:
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import string

def preprocess_query(query):
    # Download NLTK resources if not already downloaded //make sure documents are preprocess in the same way
    nltk.download('punkt')
    nltk.download('stopwords')
    english_stopwords = frozenset(stopwords.words('english'))
    corpus_stopwords = ["category", "references", "also", "external", "links",
                    "may", "first", "see", "history", "people", "one", "two",
                    "part", "thumb", "including", "second", "following",
                    "many", "however", "would", "became"]

    all_stopwords = english_stopwords.union(corpus_stopwords)
    # Initialize Porter Stemmer
    #stemmer = PorterStemmer()

    # Tokenize the query
    tokens = word_tokenize(query)

    # Remove punctuation
    tokens = [token for token in tokens if token not in string.punctuation]

    # Remove stop words
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [token for token in tokens if token.lower() not in all_stopwords]

    # Stemming
    #stemmed_tokens = [stemmer.stem(token) for token in filtered_tokens]

    # Perform additional preprocessing if needed

    return filtered_tokens

# def vectorize_query(query, inverted_index):
#     # Preprocess the query
#     preprocessed_query = preprocess_query(query)


#     # Initialize a vector for the query
#     query_vector = np.zeros(len(inverted_index.posting_locs))# to do:create vector in size of the query

#     # Calculate TF-IDF for the query terms
#     for term in preprocessed_query:

#         if term in inverted_index.posting_locs:
#             # Calculate TF (term frequency) for the query term

#             index = inverted.dictIndexTerm[term]
#             tf = preprocessed_query.count(term)/len(preprocessed_query)

#             # Calculate TF-IDF score for the query term
#             tf_idf = tf * 1

#             # Assign TF-IDF score to the corresponding dimension in the query vector
#             query_vector[index] = tf_idf

#     return query_vector
def vectorize_query(query, inverted_index):
    # Preprocess the query
    preprocessed_query = preprocess_query(query)

    dict_tokens_unq = Counter(preprocessed_query)
    # Initialize a vector for the query
    query_vector = np.zeros(len(dict_tokens_unq))# to do:create vector in size of the query
    counter = 0
    # Calculate TF-IDF for the query terms
    for term, freq in dict_tokens_unq.items():

        if inverted_index.posting_locs.get(term) is not None:
            # Calculate TF (term frequency) for the query term

            tf = freq/len(preprocessed_query)

            # Calculate TF-IDF score for the query term
            tf_idf = tf * 1

            # Assign TF-IDF score to the corresponding dimension in the query vector
            query_vector[counter] = tf_idf
            counter += 1
    return query_vector


def vectorize_documents(inverted_index,query_tokens_unq):
    # Initialize a dictionary to store document vectors
    document_vectors = {}



    counter = 0
    # Calculate TF-IDF for each document
    for term in query_tokens_unq:
        #if term not in inverted.df.keys()
        if inverted_index.df.get(term) is None:
          counter += 1
          continue
        # posting_list = inverted.read_a_posting_list(base_dir, term, bucket_name)
        posting_list = inverted.read_a_posting_list("index_text_on_jason",term, bucket_name)
        num_of_docs = len(inverted_index.document_len.items())
        print(str(num_of_docs)+"num_of_docs")
        df_of_term = inverted_index.df[term]
        idf = np.log(num_of_docs/df_of_term)  # Adding 1 to avoid division by zero
        print(idf)
        for doc_id, tf in posting_list:
            tf = tf / (inverted.document_len[doc_id])#remove 0.1
            # Calculate TF-IDF score for the term in the document
            tf_idf = tf * idf

            # If the document vector already exists, update it
            if doc_id in document_vectors:
                document_vectors[doc_id][counter] = tf_idf
            # Otherwise, create a new document vector
            else:
                document_vector = np.zeros(len(query_tokens_unq))
                document_vectors[doc_id] = document_vector
                document_vectors[doc_id][counter] = tf_idf
        counter += 1
    return document_vectors
def cosine_similarity(v1, v2):
    """Compute cosine similarity between two vectors."""
    dot_product = np.dot(v1, v2)
    norm_v1 = np.linalg.norm(v1)
    norm_v2 = np.linalg.norm(v2)
    return dot_product / (norm_v1 * norm_v2)
def MB25_similarity(inverted, dict_query_term_tf,number_to_return,k1,k2, b)
    #return top results sorted.
    query_tokens_unq = list(dict_query_term_tf.keys())
    B = 1-b +b*
    similarities = {}
    for term in query_tokens_unq:
        #if term not in inverted.df.keys()
        if inverted_index.df.get(term) is None:
          counter += 1
          continue
        posting_list = inverted.read_a_posting_list("index_text_on_jason",term, bucket_name)
        num_of_docs = len(inverted_index.document_len.items())
        df_of_term = inverted_index.df[term]
        idf = np.log(num_of_docs+1/df_of_term)  # Adding 1 to avoid division by zero
        print(idf)
        for doc_id, tf in posting_list:
            B = 1-b + b*inverted.document_len[doc_id]/inverted.avg_doc_len
            query_tf =dict_query_term_tf[term]
            bm25_value_iteration = ((k1+1)*tf)/(B*k1+tf)*idf*((k2+1)*query_tf)/(k2+query_tf)
            # If the document vector already exists, update it
            similarities[doc_id] += bm25_value_iteration
      top_results = sorted(similarities.items(), key=lambda x: x[1], reverse=True)[:number_to_return]

def search(query, inverted_index, k=10, use_cosine):
    """Search for documents based on a query using cosine similarity."""
    # Preprocess query (e.g., tokenize, remove stop words, etc.)
    # Vectorize the query

    query_vector = vectorize_query(query,inverted)
    processed_query = preprocess_query(query)
    list_tokens_unq = list(Counter(processed_query).keys())
    if use_cosine = true:
      vectorize_documents_res = vectorize_documents(inverted,list_tokens_unq)
      # Compute cosine similarity between the query vector and all document vectors
      similarities = {}
      for doc_id, doc_vector in vectorize_documents_res.items():
          similarities[doc_id] = cosine_similarity(query_vector, doc_vector)

      # Sort documents by similarity and return the top k results
      top_results = sorted(similarities.items(), key=lambda x: x[1], reverse=True)[:k]
    #use MB25
    else
      top results = MB25_similarity(inverted,Counter(processed_query),k,k1=2,k2=2,b=0.5)


    # Retrieve titles for the top results
    results_with_titles = [(doc_id, inverted_index.dict_doc_title.get(doc_id)) for doc_id, _ in top_results]
    #results_with_titles = [(doc_id, doc_titles[doc_id]) for doc_id, _ in top_results]
    return results_with_titles



test

In [ ]:
#test
# first_10_items = list(inverted.document_len.items())[:10]

# # Print the first 10 items
# print(first_10_items)

query = "genetics"
proces_query = preprocess_query(query)
print(proces_query)
query_vector = vectorize_query(query,inverted)
print(query_vector)
print(search(query,inverted))


['genetics']
[1.]


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


NotFound: 404 GET https://storage.googleapis.com/storage/v1/b/bucket-mr-project-ir-david/o/index_text_on_jason%2F32_002.bin?projection=noAcl&prettyPrint=false: No such object: bucket-mr-project-ir-david/index_text_on_jason/32_002.bin

In [ ]:
# !gsutil ls -lh $index_dst

 18.45 MiB  2024-02-19T21:17:21Z  gs://bucket-mr-project-ir-david/postings_gcp/index.pkl
TOTAL: 1 objects, 19341442 bytes (18.45 MiB)
